In [1]:
import os
import sys
sys.path.append(os.path.abspath(".."))

import numpy as np
import matplotlib.pyplot as plt
%config InlineBackend.figure_format = 'retina'  # For sharper figures, but it takes more time
from tqdm import tqdm
import scipy as sp
from copy import deepcopy 

from lisatools.utils.constants import *
from lisatools.sensitivity  import AE1SensitivityMatrix
from bbhx.waveformbuild import BBHWaveformFD
import noise_generation as noise_generation
from tools.LISASimulator import LISASimulator
from tools.time_freq_likelihood import TimeFreqLikelihood
from tools.likelihood import get_dh, get_hh, TimeFreqSNR
import tools.likelihood as likelihood

from tools.MBHB_differential_evolution import MBHB_finder

No CuPy
No CuPy or GPU PhenomHM module.
No CuPy or GPU interpolation available.
No CuPy or GPU response available.


In [2]:
Tobs = YRSID_SI/12
dt = 5.
include_T_channel = False # Set to True if you want to include the T channel in the simulation, otherwise only A and E channels will be included.

wave_gen = BBHWaveformFD(amp_phase_kwargs=dict(run_phenomd=False))
sim = LISASimulator(Tobs=Tobs, dt=dt, wave_gen=wave_gen, include_T_channel=include_T_channel)

m1 = 3e5
m2 = 1.5e5
a1 = 0.2
a2 = 0.4
dist = 5 * PC_SI * 1e9  # distance in Gpc
phi_ref = np.pi/2
f_ref = 0.0
inc = np.pi/3
lam = np.pi/1.
beta = np.pi/4.
psi = np.pi/4.
t_ref = 0.95 * Tobs
#t_ref = round(0.9 * Tobs / dt) * dt  # round to the nearest multiple of dt, to force t_ref to be a part of t_array

parameters = np.array([m1, m2, a1, a2, dist, phi_ref, f_ref, inc, lam, beta, psi, t_ref])

modes = [(2,2), (2,1), (3,3), (3,2), (4,4), (4,3)]
waveform_kwargs = dict(length=1024, direct=False, fill=True, squeeze=False, modes=modes)

data_t, data_f, f_array, t_array, sens_mat = sim(seed = 42, parameters=parameters, waveform_kwargs=waveform_kwargs)
waveform_kwargs.update(freqs=f_array)

print(sim.SNR_optimal()[0])

2876.387525169864


## Pre-Merger

In [4]:
time_before_merger = 60*60
cutoff_time = t_ref - time_before_merger
max_time = t_ref + 60*60*12

def pre_merger(gravitational_wave_data_t, time_before_merger, t_ref, t_array):
        cutoff_time = t_ref - time_before_merger
        cutoff_index = np.searchsorted(t_array, cutoff_time)
        data_t_truncated = gravitational_wave_data_t[:, :cutoff_index]
        return data_t_truncated, cutoff_index

data_t_truncated, cutoff_index =  pre_merger(data_t, time_before_merger, t_ref, t_array)

# The SNR does not depend on the distance. 
Change the guess_distance to see that the SNR is the same. The distance is calculated based on the amplitude.

In [27]:
guess_distance = dist * 1000
template = wave_gen(
    m1,
    m2,
    a1,
    a2,
    guess_distance, 
    phi_ref,
    f_ref, 
    inc,
    lam,
    beta,
    psi,
    t_ref,
    **waveform_kwargs
)
template = template[0, :2]
print(likelihood.template_snr(data_f, template, AE1SensitivityMatrix(f_array), df=sim.df))

hh = get_hh(template, AE1SensitivityMatrix(f_array), df=sim.df)
dh = get_dh(data_f, template, AE1SensitivityMatrix(f_array), df=sim.df)
amplitude = dh/hh
new_distance = guess_distance /  amplitude

print((new_distance - dist)/(PC_SI*1e9) , (new_distance-dist)*100/dist , "%" )

2876.2995595533453
0.00015291456035319145 0.0030582912070638295 %


The SNR also does not change with the time-frequency SNR with pre-merger. Change guess_distance and see that nothing changes. Also, new_distance remains the same

In [28]:
guess_distance = dist * 100
parameters_new = [
    m1*(1 + 0.001),  # Slightly change m1 to see the effect
    m2,
    a1*(1 + 0.01),  # Slightly change a1 to see the effect
    a2,
    guess_distance,
    phi_ref,
    f_ref,
    inc*(1 + 0.01),
    lam,
    beta,
    psi,
    t_ref
]
analysis = TimeFreqSNR(
    data_t_truncated,
    wave_gen=wave_gen,
    nperseg=5000,
    dt_full=dt,
    cutoff_index=cutoff_index,
    pre_merger=True
)
analysis.get_stft_of_data()
SNR, amplitude = analysis.calculate_time_frequency_SNR(*parameters_new, waveform_kwargs=waveform_kwargs)
new_distance = guess_distance /  amplitude
print((new_distance - dist)/(PC_SI*1e9) , (new_distance-dist)/dist)

-0.022924402780016243 -0.004584880556003249


# Differential Evolution Analysis

In [6]:
boundaries = {}
boundaries['TotalMass'] = [5e5, 6e5]
boundaries['MassRatio'] = [0.05, 0.999999]
boundaries['Spin1'] = [-1, 1]
boundaries['Spin2'] = [-1, 1]
boundaries['Distance'] = [1e3, 10e3] # in Mpc i.e. dL / (PC_SI * 1e6)
boundaries['Phase'] = [0.0, 2 * np.pi]
boundaries['Inclination'] = [-1, 1]
boundaries['EclipticLongitude'] = [0, 2*np.pi]
boundaries['EclipticLatitude'] = [-1, 1]
boundaries['Polarization'] = [0, np.pi]
boundaries['CoalescenceTime'] = [cutoff_time, max_time]  

In [7]:
DifferentialEvolution = MBHB_finder(
    data_t = data_t_truncated,
    wave_gen= wave_gen,
    cutoff_index=cutoff_index,
    waveform_kwargs=waveform_kwargs,
    boundaries=boundaries,
    nperseg=5000,
    dt_full= dt,
    pre_merger=True,
)
DifferentialEvolution.get_stft_of_data()

In [8]:
found_parameters, found_snr_found, function_evaluations = DifferentialEvolution.find_MBHB(
    strategy='best1exp', 
    popsize=3,
    tol=1e-1,
    maxiter=10,
    recombination=1,
    mutation=(0.7, 1),
    polish=False,
    disp=False
)

time SNR  0.28
initial guess -0.003804717613724419


/opt/anaconda3/envs/lisa_env/lib/python3.12/site-packages/scipy/optimize/_differentialevolution.py:486: UserWarning: differential_evolution: the 'workers' keyword has overridden updating='immediate' to updating='deferred'
  with DifferentialEvolutionSolver(func, bounds, args=args,


No CuPy
No CuPy or GPU PhenomHM module.
No CuPy or GPU interpolation available.
No CuPy or GPU response available.
No CuPy
No CuPy or GPU PhenomHM module.
No CuPy or GPU interpolation available.
No CuPy or GPU response available.
No CuPy
No CuPy or GPU PhenomHM module.
No CuPy or GPU interpolation available.
No CuPy or GPU response available.
No CuPy
No CuPy or GPU PhenomHM module.
No CuPy or GPU interpolation available.
No CuPy or GPU response available.
No CuPy
No CuPy or GPU PhenomHM module.
No CuPy or GPU interpolation available.
No CuPy or GPU response available.
No CuPy
No CuPy or GPU PhenomHM module.
No CuPy or GPU interpolation available.
No CuPy or GPU response available.
No CuPy
No CuPy or GPU PhenomHM module.
No CuPy or GPU interpolation available.
No CuPy or GPU response available.
No CuPy
No CuPy or GPU PhenomHM module.
No CuPy or GPU interpolation available.
No CuPy or GPU response available.
No CuPy
No CuPy or GPU PhenomHM module.
No CuPy or GPU interpolation available.


In [23]:
def transform_parameters(x):
    all_parameters = np.zeros(11)
    all_parameters[0] = x[0] + x[1]
    all_parameters[1] = x[1] / x[0]
    all_parameters[2] = x[2]
    all_parameters[3] = x[3]
    all_parameters[4] = x[4] / (PC_SI * 1e6)
    all_parameters[5] = x[5]
    all_parameters[6] = np.cos(x[7])
    all_parameters[7] = x[8]
    all_parameters[8] = np.sin(x[9])
    all_parameters[9] = x[10]
    all_parameters[10] = x[11]
    return all_parameters

In [26]:
parameters_sample = [
    'TotalMass', 'MassRatio', 'Spin1', 'Spin2', 'dL/(PC_SI * 1e6)',
    'Phase', 'cos(i)', 'lambda', 'sin(beta)', 'psi', 'tref'
]

print(f"{'Parameter':<20} {'Found':<20} {'True':<20}")
print('-' * 60)

for i, param in enumerate(parameters_sample):
    found = found_parameters[i]
    true = transform_parameters(parameters)[i]
    print(f"{param:<20} {found:<20} {true:<20}")


Parameter            Found                True                
------------------------------------------------------------
TotalMass            546591.3300411195    450000.0            
MassRatio            0.1931868759847878   0.5                 
Spin1                0.1537946378473849   0.2                 
Spin2                0.3741491336479901   0.4                 
dL/(PC_SI * 1e6)     16234.883071022541   5000.0              
Phase                5.893827068480487    1.5707963267948966  
cos(i)               -0.17082120199889383 0.5000000000000001  
lambda               1.0975563492245919   3.141592653589793   
sin(beta)            0.005292885573639161 0.7071067811865475  
psi                  0.39587524161017695  0.7853981633974483  
tref                 2500782.345529318    2498353.5229473603  


Next Steps:
- no noise
- include merger
- check for only one parameter
- tolerance smaller
- iterations bigger
- check what the variable results contain